In [31]:
import os
!git branch issue/4
!git checkout issue/4

fatal: A branch named 'issue/4' already exists.
M	storyteller/scripts/gamsung_daehwa.ipynb
Already on 'issue/4'
Your branch is up to date with 'origin/issue/4'.


In [32]:
!source ../../venv/bin/activate

In [33]:
import sys
import os

# GCP Storage 프로젝트 환경변수 설정
os.environ["GCLOUD_PROJECT"] = "wisdomify-323515"

In [34]:
# 실행한는 디렉토리 수정
sys.path.insert(0, '/workspace')

from storyteller.storyteller.utils.gcp_dataloader import GCPStorage
from storyteller.storyteller.utils.elastic_controller import ElasticController

In [35]:
gcp_storage = GCPStorage('wisdomify')
es_controller = ElasticController(
    cloud_id = "https://storyteller.es.asia-northeast3.gcp.elastic-cloud.com:9243/",
    user_id = "ArtemisDicoTiar",
    user_pw = "H^&3141592",
    index_name="wisdom2eg"
)

In [36]:
gcp_storage.download('story/elastic/감성대화', './')

Successfully created the directory: ./story/elastic/감성대화
Successfully created the directory: ./story/elastic/감성대화/Training
Successfully created the directory: ./story/elastic/감성대화/Validation


100%|██████████| 28.3M/28.3M [00:00<00:00, 97.6MB/s]



Successful download: story/elastic/감성대화/Training/감성대화말뭉치(원천데이터)_Training.zip


100%|██████████| 20.5M/20.5M [00:00<00:00, 101MB/s] 



Successful download: story/elastic/감성대화/Training/감성대화말뭉치(최종데이터)_Training.zip


100%|██████████| 3.54M/3.54M [00:00<00:00, 37.1MB/s]



Successful download: story/elastic/감성대화/Validation/감성대화말뭉치(원천데이터)_Validation.zip


100%|██████████| 2.12M/2.12M [00:00<00:00, 25.3MB/s]


Successful download: story/elastic/감성대화/Validation/감성대화말뭉치(최종데이터)_Validation.zip


In [37]:
import json

In [38]:
train = json.load(open('./story/elastic/감성대화/Training/감성대화말뭉치(최종데이터)_Training.json'))
validate = json.load(open('./story/elastic/감성대화/Validation/감성대화말뭉치(최종데이터)_Validation.json'))
total = train + validate

In [39]:
print(f"Train: {len(train)}\nValidate: {len(validate)}\nTotal: {len(total)}")

Train: 40827
Validate: 5122
Total: 45949


In [40]:
from pprint import pprint
pprint(total[0])

{'profile': {'emotion': {'emotion-id': 'S05_D02_E68',
                         'situation': ['S05', 'D02'],
                         'type': 'E68'},
             'persona': {'computer': ['C01'],
                         'human': ['A02', 'G01'],
                         'persona-id': 'A02_G01_C01'},
             'persona-id': 'Pro_03719'},
 'talk': {'content': {'HS01': '아내가 드디어 출산하게 되어서 정말 신이 나.',
                      'HS02': '아 지금 정말 신이 나.',
                      'HS03': '아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지. ',
                      'SS01': '아내분이 출산을 하시는군요. 정말 축하드려요.',
                      'SS02': '잘 된 일이네요.',
                      'SS03': '좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.'},
          'id': {'profile-id': 'Pro_03719', 'talk-id': 'Pro_03719_00016'}}}


In [41]:
data = list(map(
    lambda case: {
        'sentence': ' '.join(case['talk']['content'].values()),
        'id': case['talk']['id'],
        'dataset': '감성대화'
    },
    total
))

In [42]:
len(data)

45949

In [43]:
pprint(data[10])

{'dataset': '감성대화',
 'id': {'profile-id': 'Pro_10653', 'talk-id': 'Pro_10653_00005'},
 'sentence': '새로 간 치과에서 조금 썩기만 한 이를 임플란트를 하라고 했어. 과잉진료는 안 한다더니. 이가 조금 썩기만 했는데도 '
             '임플란트를 해야 한다고 했다는 말씀이시군요. 예전에 다니던 치과에서는 심하지 않은 충치니까 관리만 하라고 했는데 '
             '말이야. 오히려 신뢰가 떨어지네. 새 치과가 과잉 진료를 하는 것 같아 믿음이 안 가시는군요. 이 상황에서 앞으로 '
             '어떻게 하실건가요? 좀 멀어도 이사 오기 전에 살던 동네의 믿을만한 옛 치과를 가야겠어. 믿을만한 치과에서 진료가 '
             '잘 되기를 바랄게요.'}


In [45]:
es_controller.write(data, bulk=True)

ValueError: Invalid info: (bulk is False) info must be dictionary .